In [ ]:
import os
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

from src.vae_model import vae_model
from src.util import DataLoader, Pearson_correlation, AortaNormalizer
from src.visualiazation import plot_relative_error_aorta
import tensorflow as tf


os.environ["CUDA_VISIBLE_DEVICES"] = "1"

**Load Data**

In [ ]:
data_path = "../../../data/PulHypStudie_Check_npz_v2/"

In [ ]:
data_loader = DataLoader(data_path)

Eit, Y, Pig = data_loader.load_data(10)

In [ ]:
aorta_normalizer = AortaNormalizer()

In [ ]:
Y_norm = aorta_normalizer.normalize_forward(Y)
Y_true = Y[:, :, 0]

In [ ]:
plt.figure(figsize=(6, 3))
plt.hist(Y_norm.flatten(), bins=100)
plt.show()
print(np.mean(Y_norm.flatten()))
print(np.var(Y_norm.flatten()))

plt.figure(figsize=(6, 3))
plt.hist(Y_true.flatten(), bins=100)
plt.show()
print(np.mean(Y_true.flatten()))
print(np.var(Y_true.flatten()))

**Test individual model**

In [ ]:
sel_model = "src/weights/vae_model_excl10_0.weights.h5"

rel_err_config = {
    "std": True,
    "var": True,
    "mean": True,
    "s_name": None,
}

vae = vae_model()
vae.compile(optimizer=tf.keras.optimizers.Adam())
vae.load_weights(sel_model)
# apply model
_, _, z = vae.encoder.predict(Y_norm)
Y_pred = vae.decoder.predict(z)[:, :, 0]

# inverse normalization
Y_pred = aorta_normalizer.normalize_inverse(Y_pred)


mae, mse, pn = plot_relative_error_aorta(Y_true, Y_pred, **rel_err_config)

In [ ]:
Y_pred.shape

In [ ]:
from src.LaTeX_export import output_err_for_LaTeX, output_curve_for_LaTeX

In [ ]:
output_err_for_LaTeX(Y_true, Y_pred, f_name="test_results.csv")

In [ ]:
output_curve_for_LaTeX(Y_true, Y_pred, f_name="test_aorta.csv")

**Archive Code**

_Take batch size of 8 for training the final VAE network._

**Tuning parameter ...?**

**Export to LaTeX**

**Best Batch-Size**

In [ ]:
vae_models = glob("src/weights/vae_model*.weights.h5")
vae_models

In [ ]:
rel_err_config = {
    "std": True,
    "var": True,
    "mean": True,
    "s_name": None,
}

MAEs = list()
MSEs = list()
PNs = list()
for sel_model in vae_models:
    rel_err_config[
        "s_name"
    ] = f"images/rel_err{sel_model.split('/')[2].split('.')[0]}.png"
    print("load", sel_model)
    vae = vae_model()
    vae.compile(optimizer=tf.keras.optimizers.Adam())
    vae.load_weights(sel_model)
    # apply model
    _, _, z = vae.encoder.predict(Y)

    Y_pred = vae.decoder.predict(z)[:, :, 0]
    # inverse normalization
    Y_pred = aorta_normalizer.normalize_inverse(Y_pred)

    mae, mse, pn = plot_relative_error_aorta(Y_true, Y_pred, **rel_err_config)

    MAEs.append(mae)
    MSEs.append(mse)
    PNs.append(pn)


MAEs = np.array(MAEs)
MSEs = np.array(MSEs)
PNs = np.array(PNs)

In [ ]:
Y_pred

In [ ]:
Y_true

In [ ]:
print(f"{MAEs=}\n\n{MSEs=}\n\n{PNs=}")
best_choice = np.argmax(PNs)
print(
    "Best choice Batch size= ",
    np.argmin(MAEs),
    np.argmin(MSEs),
    best_choice,
    "\n\t",
    vae_models[best_choice],
)